In [1]:
print('its working')

its working


In [2]:
%cd /content/LLM-training

/content/LLM-training


In [3]:
!pip -q install -U transformers datasets accelerate peft trl bitsandbytes
print('success')

success


In [ ]:
import os
os.environ["HF_TOKEN"] = ""

In [4]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTConfig, SFTTrainer


In [5]:
base = "https://huggingface.co/datasets/kiddothe2b/contract-nli/resolve/refs%2Fconvert%2Fparquet"

data_files = {
    "train":      f"{base}/contractnli_a/train/0000.parquet",
    "validation": f"{base}/contractnli_a/validation/0000.parquet",
    "test":       f"{base}/contractnli_a/test/0000.parquet",
}



In [6]:
ds = load_dataset("parquet", data_files=data_files)
print(ds["train"][0])
print(ds["train"][1])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'premise': '2.3 Provided that the Recipient has a written agreement with the following persons or entities requiring them to treat the Confidential Information in accordance with this Agreement, the Recipient may disclose the Confidential Information to: 2.3.1  Any other party with the Discloser’s prior written consent; and 2.3.2 the Recipient’s employees, officials, representatives and agents who have a strict need to know the contents of the Confidential Information, and employees, officials, representatives and agents of any legal entity that it controls, controls it, or with which it is under common control, who have a similar need to know the contents of the Confidential Information, provided that, for these purposes a controlled legal entity means:', 'hypothesis': "Receiving Party shall not reverse engineer any objects which embody Disclosing Party's Confidential Information.", 'label': 2}
{'premise': '5. All Confidential Information in any form and any medium, including all cop

In [7]:
# print label feature info
print(ds["train"].features["label"])

ClassLabel(names=['contradiction', 'entailment', 'neutral'])


In [8]:
label_map = {0: "entailment", 1: "contradiction", 2: "neutral"}

def format_example(ex):
    label = label_map[int(ex["label"])]
    return {
        "text": (
            "### Instruction:\n"
            "Classify the hypothesis using the contract text as entailment, contradiction, or not_mentioned.\n\n"
            f"### Contract:\n{ex['premise']}\n\n"
            f"### Hypothesis:\n{ex['hypothesis']}\n\n"
            "### Label:\n"
            f"{label}"
        )
    }

train_ds = ds["train"].shuffle(seed=42).select(range(min(1200, len(ds["train"])))).map(
    format_example, remove_columns=ds["train"].column_names
)
eval_ds  = ds["validation"].shuffle(seed=7).select(range(min(200, len(ds["validation"])))).map(
    format_example, remove_columns=ds["validation"].column_names
)

print(train_ds[0]["text"][:700])


### Instruction:
Classify the hypothesis using the contract text as entailment, contradiction, or not_mentioned.

### Contract:
The term "Evaluation Materials" includes information furnished to you orally or in writing (whatever the form or storage medium) or gathered by inspection,and regardless of whether such information is specifically identified as "confidential." 

### Hypothesis:
Confidential Information shall only include technical information.

### Label:
neutral


In [9]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token
tokenizer.model_max_length = 512

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

In [10]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)


In [13]:
out_dir = "contractnli-a-qlora-tinyllama"

args = SFTConfig(
    output_dir=out_dir,
    max_steps=120,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=60,
    eval_strategy="steps",
    eval_steps=60,
    fp16=False,
    bf16=False,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    report_to="none",
)


In [14]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    peft_config=peft_config,
    processing_class=tokenizer,
)

trainer.train()


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/1200 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1200 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
60,1.029950,1.119884
120,0.979597,1.097783


TrainOutput(global_step=120, training_loss=1.136958940823873, metrics={'train_runtime': 440.505, 'train_samples_per_second': 2.179, 'train_steps_per_second': 0.272, 'total_flos': 1627226664763392.0, 'train_loss': 1.136958940823873})

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
LORA_DIR = "/content/LLM-training/contractnli-a-qlora-tinyllama/checkpoint-120"

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

# 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# load base model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
)

# load LoRA adapter
model = PeftModel.from_pretrained(model, LORA_DIR)
model.eval()

def classify(contract, hypothesis):
    prompt = (
        "### Instruction:\n"
        "Classify the hypothesis using the contract text as entailment, contradiction, or not_mentioned.\n\n"
        f"### Contract:\n{contract}\n\n"
        f"### Hypothesis:\n{hypothesis}\n\n"
        "### Label:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            do_sample=False,
            temperature=0.0,
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text.split("### Label:")[-1].strip()

# ---- Test case ----
contract = "This Agreement shall be governed by the laws of the State of California."
hypothesis = "The contract is governed by the laws of the State of New York."


print("Prediction:", classify(contract, hypothesis))


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Prediction: contradiction
